In [1]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#load the claims data

data=pd.read_csv(r"final_data.csv",header=0)
data_bk=data.copy()
data.head()

,Location,MinTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Temp3pm,RainToday,RainTomorrow,Season
0,0.041667,0.535620,0.021898,0.866667,0.527778,0.866667,0.933333,0.512821,0.545455,0.666667,0.212121,0.253102,0.502439,0,0,0.4
1,0.041667,0.377309,0.000000,0.933333,0.527778,0.400000,1.000000,0.102564,0.500000,0.356322,0.242424,0.325062,0.563415,0,0,0.4
2,0.041667,0.522427,0.000000,1.000000,0.555556,0.866667,1.000000,0.487179,0.590909,0.287356,0.292929,0.250620,0.536585,0,0,0.4
3,0.041667,0.424802,0.000000,0.266667,0.250000,0.600000,0.000000,0.282051,0.204545,0.367816,0.151515,0.498759,0.617073,0,0,0.4
4,0.041667,0.643799,0.036496,0.866667,0.486111,0.066667,0.466667,0.179487,0.454545,0.793103,0.323232,0.330025,0.695122,0,0,0.4


In [3]:
data.columns

Index(['Location', 'MinTemp', 'Rainfall', 'WindGustDir', 'WindGustSpeed',
       'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Temp3pm', 'RainToday',
       'RainTomorrow', 'Season'],
      dtype='object')

In [4]:
indepvar=[]
for col in data.columns:
    if col!='RainTomorrow':
        indepvar.append(col)
depvar='RainTomorrow'
x=data[indepvar]
y=data[depvar]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((96930, 15), (41542, 15), (96930,), (41542,))

VotingClassifier

In [14]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    roc_auc_score,
)

# Create base models
catboost_model = CatBoostClassifier(verbose=0)
xgboost_model = xgb.XGBClassifier(verbosity=0)


# Create a VotingClassifier with weights
ensemble_model_weighted = VotingClassifier(estimators=[('catboost', catboost_model), ('xgboost', xgboost_model)],
                                           voting='soft', weights=[2, 1])


# Train the ensemble model with weights
ensemble_model_weighted.fit(x_train, y_train)

# Evaluate the ensemble model with weights
y_pred_weighted = ensemble_model_weighted.predict(x_test)

# Calculate evaluation metrics
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
precision_weighted = precision_score(y_test, y_pred_weighted)
recall_weighted = recall_score(y_test, y_pred_weighted)
f1_weighted = f1_score(y_test, y_pred_weighted)
balanced_accuracy_weighted = balanced_accuracy_score(y_test, y_pred_weighted)
matthews_corr_weighted = matthews_corrcoef(y_test, y_pred_weighted)
roc_auc_weighted = roc_auc_score(y_test, ensemble_model_weighted.predict_proba(x_test)[:, 1])

# Create a DataFrame with the metrics
results_df_weighted = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Balanced Accuracy', 'Matthews Corrcoef', 'ROC AUC'],
    'Value': [accuracy_weighted, precision_weighted, recall_weighted, f1_weighted, balanced_accuracy_weighted,
              matthews_corr_weighted, roc_auc_weighted]
})


# Print the metrics
print("Ensemble Model Metrics (Weighted):")
print(results_df_weighted)


Ensemble Model Metrics (Weighted):
              Metric     Value
0           Accuracy  0.860310
1          Precision  0.731190
2             Recall  0.491628
3           F1 Score  0.587943
4  Balanced Accuracy  0.722838
5  Matthews Corrcoef  0.522205
6            ROC AUC  0.876369


Hyperparameter tuning

In [15]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef,
    roc_auc_score,
)
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# Assuming df_resampled is your DataFrame
# Split the data into features (X) and target variable (y)

# Create base models
catboost_model = CatBoostClassifier(verbose=0)
xgboost_model = XGBClassifier(verbosity=0)

# Create a VotingClassifier with weights
ensemble_model_weighted = VotingClassifier(estimators=[('catboost', catboost_model), ('xgboost', xgboost_model)],
                                           voting='soft', weights=[2, 1])

# Hyperparameter tuning
param_grid = {
    'catboost__depth': [4, 6, 8],
    'xgboost__max_depth': [3, 5, 7],
}

grid_search = GridSearchCV(ensemble_model_weighted, param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
matthews_corr = matthews_corrcoef(y_test, y_pred)
roc_auc = roc_auc_score(y_test, best_model.predict_proba(x_test)[:, 1])

# Create a DataFrame with the metrics and best parameters
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Balanced Accuracy', 'Matthews Corrcoef', 'ROC AUC'],
    'Value': [accuracy, precision, recall, f1, balanced_accuracy, matthews_corr, roc_auc]
})

# Add the best parameters to the DataFrame
results_df.loc[len(results_df)] = ['Best Parameters', grid_search.best_params_]

# Save the DataFrame to a CSV file
results_df.to_csv('hypertuning_metrics.csv', index=False)

# Print the metrics and best parameters
print(results_df)


              Metric                                            Value
0           Accuracy                                         0.860262
1          Precision                                         0.724434
2             Recall                                         0.501366
3           F1 Score                                         0.592603
4  Balanced Accuracy                                         0.726438
5  Matthews Corrcoef                                         0.524245
6            ROC AUC                                         0.877553
7    Best Parameters  {'catboost__depth': 8, 'xgboost__max_depth': 7}


KFold Cross-Validation

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# Assuming df_resampled is your DataFrame

# Separate features and target variable
X = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']

# Modify the main loop to use the best model with specified hyperparameters and cross-validation
def cross_validation_with_best_model(X, y, num_folds, best_model):
    all_fold_metrics = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for fold_num, (train_index, test_index) in enumerate(kf.split(X)):
        fold_metrics = {}

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)

        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        fold_metrics['accuracy'] = accuracy_score(y_test, y_pred)
        fold_metrics['precision'] = precision_score(y_test, y_pred)
        fold_metrics['recall'] = recall_score(y_test, y_pred)
        fold_metrics['f1'] = f1_score(y_test, y_pred)
        fold_metrics['roc_auc'] = roc_auc_score(y_test, y_prob)
   
        all_fold_metrics.append(fold_metrics)

        print(f'Fold {fold_num + 1}: Accuracy = {fold_metrics["accuracy"]}')

    return all_fold_metrics

# Define k-fold cross-validation settings
kf_5 = KFold(n_splits=5, shuffle=True, random_state=42)
kf_10 = KFold(n_splits=10, shuffle=True, random_state=42)
kf_15 = KFold(n_splits=15, shuffle=True, random_state=42)
kf_20 = KFold(n_splits=20, shuffle=True, random_state=42)

# Iterate through different numbers of folds
kfolds = [kf_5]# kf_10, kf_15, kf_20

# Create an empty DataFrame to store mean metrics
mean_metrics_df = pd.DataFrame()

for num_folds, kf in zip([5, 10, 15, 20], kfolds):
    print(f'Number of Folds: {num_folds}')

    # Perform cross-validation for each fold
    fold_metrics = cross_validation_with_best_model(X, y, num_folds, best_model)

    # Calculate and print mean of metrics for all folds
    mean_metrics = {metric: np.mean([fold[metric] for fold in fold_metrics]) for metric in fold_metrics[0]}
    mean_metrics['k_value'] = num_folds
    # Append mean metrics to the DataFrame
    mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)

    # Print mean metrics for all folds
    print('Mean Metrics for All Folds:')
    for metric, value in mean_metrics.items():
        print(f'{metric}: {value}')
    print()



Number of Folds: 5
Fold 1: Accuracy = 0.8602996930853944
Fold 2: Accuracy = 0.860588553890594
Fold 3: Accuracy = 0.8614862425074024
Fold 4: Accuracy = 0.8601141041380804
Fold 5: Accuracy = 0.8613779157940348
Mean Metrics for All Folds:
accuracy: 0.8607733018831011
precision: 0.7375129754487006
recall: 0.49936797647832476
f1: 0.5955004772990329
roc_auc: 0.8783341638922428
k_value: 5



C:\Users\Maha\AppData\Local\Temp\ipykernel_19976\1928167893.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
 
)


# Separate features and target variable
X = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']


# Modify the main loop to use LightGBM with specified hyperparameters and cross-validation
def voting_cross_validation(X, y, num_folds):
    all_fold_metrics = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for fold_num, (train_index, test_index) in enumerate(kf.split(X)):
        fold_metrics = {}

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)

        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        fold_metrics['accuracy'] = accuracy_score(y_test, y_pred)
        fold_metrics['precision'] = precision_score(y_test, y_pred)
        fold_metrics['recall'] = recall_score(y_test, y_pred)
        fold_metrics['f1'] = f1_score(y_test, y_pred)
       
        fold_metrics['roc_auc'] = roc_auc_score(y_test, y_prob)
       

        all_fold_metrics.append(fold_metrics)

        print(f'Fold {fold_num + 1}: Accuracy = {fold_metrics["accuracy"]}')

    return all_fold_metrics

# Define k-fold cross-validation settings
kf_5 = KFold(n_splits=5, shuffle=True, random_state=42)
kf_10 = KFold(n_splits=10, shuffle=True, random_state=42)
kf_15 = KFold(n_splits=15, shuffle=True, random_state=42)
kf_20 = KFold(n_splits=20, shuffle=True, random_state=42)

# Iterate through different numbers of folds
kfolds = [kf_10]# kf_10, kf_15, kf_20


for num_folds, kf in zip([10], kfolds):
    print(f'Number of Folds: {num_folds}')

    # Perform cross-validation for each fold
    fold_metrics = voting_cross_validation(X, y, num_folds)

    # Calculate and print mean of metrics for all folds
    mean_metrics = {metric: np.mean([fold[metric] for fold in fold_metrics]) for metric in fold_metrics[0]}
    mean_metrics['k_value'] = num_folds
    # Append mean metrics to the DataFrame
    mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)

    # Print mean metrics for all folds
    print('Mean Metrics for All Folds:')
    for metric, value in mean_metrics.items():
        print(f'{metric}: {value}')
    print()

# Save mean metrics to a CSV file


Number of Folds: 10
Fold 1: Accuracy = 0.8586799537839399
Fold 2: Accuracy = 0.8611351819757366
Fold 3: Accuracy = 0.8623528562143424
Fold 4: Accuracy = 0.8597530150935221
Fold 5: Accuracy = 0.8617028959341374
Fold 6: Accuracy = 0.8620639849786957
Fold 7: Accuracy = 0.8580197876796418
Fold 8: Accuracy = 0.8635083411569293
Fold 9: Accuracy = 0.8565032136924966
Fold 10: Accuracy = 0.8661804000866614
Mean Metrics for All Folds:
accuracy: 0.8609899630596104
precision: 0.7372954574477745
recall: 0.5013215202750827
f1: 0.5967640021287994
roc_auc: 0.8795187462199967
k_value: 10



C:\Users\Maha\AppData\Local\Temp\ipykernel_19976\2446411022.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)


In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)


# Separate features and target variable
X = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']


# Modify the main loop to use LightGBM with specified hyperparameters and cross-validation
def voting_cross_validation(X, y, num_folds):
    all_fold_metrics = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for fold_num, (train_index, test_index) in enumerate(kf.split(X)):
        fold_metrics = {}

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)

        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        fold_metrics['accuracy'] = accuracy_score(y_test, y_pred)
        fold_metrics['precision'] = precision_score(y_test, y_pred)
        fold_metrics['recall'] = recall_score(y_test, y_pred)
        fold_metrics['f1'] = f1_score(y_test, y_pred)
        fold_metrics['roc_auc'] = roc_auc_score(y_test, y_prob)
       

        all_fold_metrics.append(fold_metrics)

        print(f'Fold {fold_num + 1}: Accuracy = {fold_metrics["accuracy"]}')

    return all_fold_metrics

# Define k-fold cross-validation settings
kf_5 = KFold(n_splits=5, shuffle=True, random_state=42)
kf_10 = KFold(n_splits=10, shuffle=True, random_state=42)
kf_15 = KFold(n_splits=15, shuffle=True, random_state=42)
kf_20 = KFold(n_splits=20, shuffle=True, random_state=42)

# Iterate through different numbers of folds
kfolds = [kf_15]# kf_10, kf_15, kf_20


for num_folds, kf in zip([15], kfolds):
    print(f'Number of Folds: {num_folds}')

    # Perform cross-validation for each fold
    fold_metrics = voting_cross_validation(X, y, num_folds)

    # Calculate and print mean of metrics for all folds
    mean_metrics = {metric: np.mean([fold[metric] for fold in fold_metrics]) for metric in fold_metrics[0]}
    mean_metrics['k_value'] = num_folds
    # Append mean metrics to the DataFrame
    mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)

    # Print mean metrics for all folds
    print('Mean Metrics for All Folds:')
    for metric, value in mean_metrics.items():
        print(f'{metric}: {value}')
    print()

# Save mean metrics to a CSV file


Number of Folds: 15
Fold 1: Accuracy = 0.8611351819757366
Fold 2: Accuracy = 0.8596187175043327
Fold 3: Accuracy = 0.8623266897746967
Fold 4: Accuracy = 0.8642764298093587
Fold 5: Accuracy = 0.8636265164644714
Fold 6: Accuracy = 0.8623266897746967
Fold 7: Accuracy = 0.8636265164644714
Fold 8: Accuracy = 0.8590618567869137
Fold 9: Accuracy = 0.8656700249160437
Fold 10: Accuracy = 0.8593868486621168
Fold 11: Accuracy = 0.8615534611634709
Fold 12: Accuracy = 0.8613367999133354
Fold 13: Accuracy = 0.8584118730365075
Fold 14: Accuracy = 0.8598201711623876
Fold 15: Accuracy = 0.8643700574152313
Mean Metrics for All Folds:
accuracy: 0.8617698556549181
precision: 0.7408367323682826
recall: 0.501898065683125
f1: 0.5983397918454697
roc_auc: 0.8798074135442577
k_value: 15



C:\Users\Maha\AppData\Local\Temp\ipykernel_19976\4011020265.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)


In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


# Separate features and target variable
X = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']


# Modify the main loop to use LightGBM with specified hyperparameters and cross-validation
def voting_cross_validation(X, y, num_folds):
    all_fold_metrics = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for fold_num, (train_index, test_index) in enumerate(kf.split(X)):
        fold_metrics = {}

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        best_model.fit(X_train, y_train)

        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        fold_metrics['accuracy'] = accuracy_score(y_test, y_pred)
        fold_metrics['precision'] = precision_score(y_test, y_pred)
        fold_metrics['recall'] = recall_score(y_test, y_pred)
        fold_metrics['f1'] = f1_score(y_test, y_pred)
        fold_metrics['roc_auc'] = roc_auc_score(y_test, y_prob)

        all_fold_metrics.append(fold_metrics)

        print(f'Fold {fold_num + 1}: Accuracy = {fold_metrics["accuracy"]}')

    return all_fold_metrics

# Define k-fold cross-validation settings
kf_5 = KFold(n_splits=5, shuffle=True, random_state=42)
kf_10 = KFold(n_splits=10, shuffle=True, random_state=42)
kf_15 = KFold(n_splits=15, shuffle=True, random_state=42)
kf_20 = KFold(n_splits=20, shuffle=True, random_state=42)

# Iterate through different numbers of folds
kfolds = [kf_20]# kf_10, kf_15, kf_20


for num_folds, kf in zip([20], kfolds):
    print(f'Number of Folds: {num_folds}')

    # Perform cross-validation for each fold
    fold_metrics = voting_cross_validation(X, y, num_folds)

    # Calculate and print mean of metrics for all folds
    mean_metrics = {metric: np.mean([fold[metric] for fold in fold_metrics]) for metric in fold_metrics[0]}
    mean_metrics['k_value'] = num_folds
    # Append mean metrics to the DataFrame
    mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)

    # Print mean metrics for all folds
    print('Mean Metrics for All Folds:')
    for metric, value in mean_metrics.items():
        print(f'{metric}: {value}')
    print()

# Save mean metrics to a CSV file


Number of Folds: 20
Fold 1: Accuracy = 0.8610629693818602
Fold 2: Accuracy = 0.8591854419410745
Fold 3: Accuracy = 0.857163489312536
Fold 4: Accuracy = 0.8653957250144425
Fold 5: Accuracy = 0.8658290005777007
Fold 6: Accuracy = 0.8587521663778163
Fold 7: Accuracy = 0.858318890814558
Fold 8: Accuracy = 0.8578856152512998
Fold 9: Accuracy = 0.8612073945696129
Fold 10: Accuracy = 0.8588965915655691
Fold 11: Accuracy = 0.8603408434430965
Fold 12: Accuracy = 0.8642403235124205
Fold 13: Accuracy = 0.8593095478838654
Fold 14: Accuracy = 0.8574317492416582
Fold 15: Accuracy = 0.8665318503538928
Fold 16: Accuracy = 0.8614762386248737
Fold 17: Accuracy = 0.8575761952910588
Fold 18: Accuracy = 0.8575761952910588
Fold 19: Accuracy = 0.8646540517116856
Fold 20: Accuracy = 0.8691318792431028
Mean Metrics for All Folds:
accuracy: 0.8610983079701592
precision: 0.73776087706573
recall: 0.5011692879722266
f1: 0.5967745722906659
roc_auc: 0.8795928385280425
k_value: 20



C:\Users\Maha\AppData\Local\Temp\ipykernel_19976\3703833889.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_metrics_df = mean_metrics_df.append(mean_metrics, ignore_index=True)


Metrics for k=5,10,15,20

In [20]:
mean_metrics_df

,accuracy,precision,recall,f1,roc_auc,k_value
0,0.860773,0.737513,0.499368,0.595500,0.878334,5.0
1,0.860990,0.737295,0.501322,0.596764,0.879519,10.0
2,0.861770,0.740837,0.501898,0.598340,0.879807,15.0
3,0.861098,0.737761,0.501169,0.596775,0.879593,20.0
